In [ ]:
import os
import sys
import numpy as np

import jax
import jax.numpy as jnp
from jax import random
from jax import grad, vmap, pmap, jit
from functools import partial

from utils import *

%config IPCompleter.use_jedi = False

In [ ]:
# Could ask students to write this function
def train_validation_split(X, y, ratio):
    dataset_size = len(X)
    label_size = len(y)
    assert dataset_size == label_size
    assert ratio <= 1 and ratio > 0
    
    ############################################
    # Split X and y based on the inputted ratio.
    # Make sure the label is not messed up.
    ############################################
    # TODO

    return train_X, train_y, validation_X, validation_y

In [ ]:
X_train, y_train, X_test, y_test = loading('.')
X_train_scaled = jnp.array(X_train / 255.)
X_test_scaled = jnp.array(X_test / 255.)
y_train_hot = jnp.squeeze(jax.nn.one_hot(y_train, num_classes=10))
y_valid_hot = jnp.squeeze(jax.nn.one_hot(y_test, num_classes=10))
X_train_standardized =\
  jax.nn.standardize(x = X_train_scaled,
                     mean = jnp.array([0.4914, 0.4822, 0.4465]),
                     variance = jnp.array([0.2023**2, 0.1994**2, 0.2010**2]))
X_test_standardized = \
  jax.nn.standardize(x = X_test_scaled,
                     mean = jnp.array([0.4914, 0.4822, 0.4465]),
                     variance = jnp.array([0.2023**2, 0.1994**2, 0.2010**2]))
train_X, train_y, valid_X, valid_y =\
  train_validation_split(X_train_standardized, y_train, 0.2)


##########################################################################
# Please tune the following 4 p's to increase your accuracy.
# You can also choose a combinatino of augmentation you like.
##########################################################################
# TODO
def contamination_factory(X, key):
    batch_size = len(X)
    
    # Random cropping with padding
    key, _ = random.split(key)
    prob = random.bernoulli(key=key, p=0.3, shape=[batch_size])
    X = jigsaw_jit(X, prob)

    #Random rotation by 90 degree
    key, _ = random.split(key)
    prob = random.bernoulli(key=key, p=0.3, shape=[batch_size])
    # X = rotation_jit(X, prob)
    
    # Random horizontal flip
    key, _ = random.split(key)
    prob = random.bernoulli(key=key, p=0.3, shape=[batch_size])
    X = horizontal_flip_jit(X, prob)
    
    #Random vertical flip
    key, _ = random.split(key)
    prob = random.bernoulli(key=key, p=0.3, shape=[batch_size])
    # X = vertocal_flip_jit(X, prob)

    return X

In [ ]:
label_dict = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}


sample_size = 30
indices = np.arange(len(train_X))
random_idx = np.random.choice(indices, size=(sample_size))

sample_X = train_X[random_idx]
sample_y = train_y[random_idx]

sample_label_dict = [label_dict.get(i) for i in sample_y]

plot_samples(
    X=sample_X,
    y=sample_label_dict,
    num_rows = 5, num_columns = 6,
    title = "Lucky Samples"
)

sample_X_contaminated = contamination_factory(
    sample_X, key=random.PRNGKey(0)
)

plot_samples(
    X=sample_X_contaminated,
    y=sample_label_dict,
    num_rows = 5, num_columns = 6,
    title = "Contaminated Samples"
)